# Dyamic Lead Gen - Proof of Concept

Set up agentic teams.

Example data.

Trace outcomes.


To start, I am just creating multi-step form presentation part and the scoring part of the flow. These are the most important to get right.

1. QuestionPresentation Team
    - QuestionSelector Agent - Determines next best question(s) based on prior answers and remaining required ones
    - Phrasing Agent - Rewrites questions dynamically (based on tone, audience, fatigue signals)
    - Engagement Agent - Detects abandonment risk (e.g. too many skips, inactivity) and uses motivational cues

2. LeadScoreResponse Team
    - LeadScoring Agent - Applies marketer rubric + inferred logic to classify lead as:
    ✅ Definite Yes, ⚠️ Needs Review, ❌ Definite No
    - Completion Agent: Custom summary & CTA message rendered on thank-you page
    - FollowupAgent: Generates personalized email for "Yes" and "Needs Review" leads



In [2]:
# imports

from typing import Any, Callable, List, Optional, TypedDict, Union

from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.runnables import Runnable
from langchain_core.tools import BaseTool
from langchain_openai import ChatOpenAI

from langgraph.graph import END, StateGraph

In [5]:
# helper functions

def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}


def create_agent(
    llm: ChatOpenAI,
    tools: list,
    system_prompt: str,
) -> str:
    """Create a function-calling agent and add it to the graph."""
    system_prompt += ("\nWork autonomously according to your specialty, using the tools available to you."
    " Do not ask for clarification."
    " Your other team members (and other teams) will collaborate with you with their own specialties."
    " You are chosen for a reason!")
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent = create_openai_functions_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor


def create_team_supervisor(llm: ChatOpenAI, system_prompt, members) -> str:
    """An LLM-based router."""
    options = ["FINISH"] + members
    function_def = {
        "name": "route",
        "description": "Select the next role.",
        "parameters": {
            "title": "routeSchema",
            "type": "object",
            "properties": {
                "next": {
                    "title": "Next",
                    "anyOf": [
                        {"enum": options},
                    ],
                },
            },
            "required": ["next"],
        },
    }
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            MessagesPlaceholder(variable_name="messages"),
            (
                "system",
                "Given the conversation above, who should act next?"
                " Or should we FINISH? Select one of: {options}",
            ),
        ]
    ).partial(options=str(options), team_members=", ".join(members))
    return (
        prompt
        | llm.bind_functions(functions=[function_def], function_call="route")
        | JsonOutputFunctionsParser()
    )

In [ ]:
# Load environment variables from file
import os
from dotenv import load_dotenv

dotenv_path = os.path.join(os.getcwd(), ".env")
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path)
else:
    raise FileNotFoundError(f".env file not found at {dotenv_path}")

# Validate required environment variables
REQUIRED_ENV_VARS = ["LANGCHAIN_API_KEY", "OPENAI_API_KEY"]
for env_var in REQUIRED_ENV_VARS:
    assert os.getenv(env_var) is not None, f"Missing required environment variable: {env_var}"


In [ ]:
# Set environment variables for LangSmith
from uuid import uuid4

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"POC JKWB - {uuid4().hex[0:8]}"

In [ ]:
# # Set up connection to Supabase
# import supabase
# from supabase import create_client, Client

# # Define Supabase connection constants
# SUPABASE_URL_ENV = "SUPABASE_URL"
# SUPABASE_KEY_ENV = "SUPABASE_SERVICE_KEY"

# SUPABASE_URL = os.getenv(SUPABASE_URL_ENV)
# SUPABASE_SERVICE_KEY = os.getenv(SUPABASE_KEY_ENV)

# assert SUPABASE_URL is not None, f"Missing required environment variable: {SUPABASE_URL_ENV}"
# assert SUPABASE_SERVICE_KEY is not None, f"Missing required environment variable: {SUPABASE_KEY_ENV}"

# # Initialize Supabase client
# supabase_client: Client = create_client(SUPABASE_URL, SUPABASE_SERVICE_KEY)

# # Example: List tables in the public schema (for verification)
# try:
#     response = supabase_client.table("pg_tables").select("*").eq("schemaname", "public").execute()
#     if hasattr(response, "data"):
#         table_names = [row["tablename"] for row in response.data]
#         print("Supabase public tables:", table_names)
#     else:
#         print("Could not retrieve table list from Supabase.")
# except Exception as e:
#     print(f"Error connecting to Supabase or listing tables: {e}")


In [ ]:
# tools

from typing import Annotated, List, Tuple, Union
from langchain_core.tools import tool

# @tool
# def retrieve_information(
#     query: Annotated[str, "query to ask the retrieve information tool"]
#     ):
#   """Use Retrieval Augmented Generation to retrieve information about student loan policies"""
#   return rag_graph.invoke({"question" : query})

from langchain_core.tools import tool
import json
from typing import Annotated, Any

@tool
def retrieve_and_parse_json(
    file_path: Annotated[str, "Path to the JSON file to retrieve and parse"]
) -> Any:
    """
    Retrieve and parse information from a JSON file.

    Args:
        file_path (str): The path to the JSON file.

    Returns:
        Any: The parsed JSON data.

    Raises:
        FileNotFoundError: If the file does not exist.
        json.JSONDecodeError: If the file is not valid JSON.
        AssertionError: If file_path is not a string or is empty.
    """
    assert isinstance(file_path, str) and file_path.strip(), "file_path must be a non-empty string"
    try:
        with open(file_path, "r", encoding="utf-8") as json_file:
            data = json.load(json_file)
        return data
    except FileNotFoundError as fnf_error:
        raise FileNotFoundError(f"JSON file not found: {file_path}") from fnf_error
    except json.JSONDecodeError as json_error:
        raise json.JSONDecodeError(f"Invalid JSON in file: {file_path}", doc=json_error.doc, pos=json_error.pos)


In [ ]:
# team state

import functools
import operator

from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_openai.chat_models import ChatOpenAI
import functools

class QuestionPresentationTeamState(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]
    team_members: List[str]
    next: str

In [9]:
llm_supervisor = ChatOpenAI(model="gpt-4o-mini")
llm_agent = ChatOpenAI(model="gpt-4o-nano")

In [ ]:
# create agents and nodes

# question selection
QUESTION_AGENT_PROMPT = """You are a marketing expert.
You will pick the best 1-3 questions to ask next. You will consider the answers to the questions already asked, information known about the user from URL query parameters, marketing expert knowledge and the body of possible remaining questions, both required and optional.
"""

question_agent = create_agent(
    llm_agent,
    [], # get questions from data
    QUESTION_AGENT_PROMPT,
)

question_node = functools.partial(agent_node, agent=question_agent, name="Questions")


# phrasing of selected questions
# this might be best as edge and regular node
PHRASING_AGENT_PROMPT = """You are a marketing expert.
You will examine the questions provided and phrase them in a friendly way to elicit as truthful and complete response as possible.
"""

phrasing_agent = create_agent(
    llm_agent,
    [],
    PHRASING_AGENT_PROMPT,
)

phrasing_node = functools.partial(agent_node, agent=phrasing_agent, name="Phrasing")


# check if user still engaged..
ENGAGEMENT_AGENT_PROMPT = """You are a marketing expert.
You will insert some useful and encouraging information for the user into the titles, text paragraphs, and submit button on the page. You seek to ensure the user completes all of the questions and learns about the valueable product or service offered by the business.
"""

engagement_agent = create_agent(
    llm_agent,
    [],
    ENGAGEMENT_AGENT_PROMPT,
)

engagement_node = functools.partial(agent_node, agent=engagement_agent, name="Engagement")




In [ ]:
# create supervisor

questions_supervisor_agent = create_team_supervisor(
    llm_supervisor,
    ("You are a supervisor tasked with managing a conversation between the"
    " following workers: Questions, Phrasing, Engagement. Given the following user request,"
    " pick the next set of questions and respond with the worker to act next."
    " Each worker will perform a task and respond with their results and status."
    " You should never ask your team to do anything beyond picking a question, rephrasing a question, or checking engagement."
    " When finished, respond with FINISH."),
    ["Questions", "Phrasing", "Engagement"],
)

In [ ]:
questions_graph = StateGraph(QuestionPresentationTeamState)

questions_graph.add_node("Questions", question_node)
# questions_graph.add_node("LoanRetriever", research_node)
questions_graph.add_node("supervisor", questions_supervisor_agent)

In [ ]:
# lead score agent
SCORING_AGENT_PROMPT = """You are a marketing expert.
You will examine the questions provided and phrase them in a friendly way to elicit as truthful and complete response as possible.
"""

scoring_agent = create_agent(
    llm_agent,
    [],
    SCORING_AGENT_PROMPT,
)

scoring_node = functools.partial(agent_node, agent=scoring_agent, name="LeadScoring")


# completion message agent
# todo store results in db
RESULT_AGENT_PROMPT = """You are a marketing expert.
You will examine the questions provided and phrase them in a friendly way to elicit as truthful and complete response as possible.
"""

result_agent = create_agent(
    llm_agent,
    [],
    RESULT_AGENT_PROMPT,
)

result_node = functools.partial(agent_node, agent=result_agent, name="ResultMessage")


# follow up message agent
FOLLOWUP_AGENT_PROMPT = """You are a marketing expert.
You will examine the questions provided and phrase them in a friendly way to elicit as truthful and complete response as possible.
"""

followup_agent = create_agent(
    llm_agent,
    [],
    FOLLOWUP_AGENT_PROMPT,
)

result_node = functools.partial(agent_node, agent=followup_agent, name="Followup")


In [ ]:
class FollowupTeamState(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]
    team_members: List[str]
    next: str

In [ ]:
# create supervisor

followup_supervisor_agent = create_team_supervisor(
    llm_supervisor,
    ("You are a supervisor tasked with managing a conversation between the"
    " following workers: ResultMessage, Followup. Given the following user request,"
    " pick the next set of questions and respond with the worker to act next."
    " Each worker will perform a task and respond with their results and status."
    " You should never ask your team to do anything beyond picking a question, rephrasing a question, or checking engagement."
    " When finished, respond with FINISH."),
    ["Questions", "Phrasing", "Engagement"],
)